In [1]:
#import libraries
import numpy as np
import pandas as pd
import os,sys
import matplotlib.pyplot as plt
import librosa
import librosa.display
from tqdm import tqdm
import IPython.display as ipd
from IPython.core.display import display
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation, Softmax,BatchNormalization,GlobalAveragePooling2D,Average
from tensorflow.keras.utils import to_categorical
from keras.models import Model, Input

**Creating an Ensemble model from Mel spectrogram**

In [2]:
def create_keras_model_mel_1(model_input, num_classes):
    x = Conv2D(64, kernel_size=3, activation="relu")(model_input)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(128, kernel_size=3, activation="relu")(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(256, kernel_size=3, activation="relu")(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(256, kernel_size=3, activation="relu")(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Flatten()(x)
    x = Dense(256, activation = 'relu')(x)
    x = Dropout(0.7)(x)
    x = Dense(num_classes, activation="softmax")(x)
    model = Model(model_input, x, name='model_1')
    
    return model

In [3]:
def create_keras_model_mel_2(model_input, num_classes):

    x = Conv2D(64, kernel_size=(3, 3), activation='relu')(model_input)
    x = BatchNormalization()(x)

    x = MaxPooling2D(pool_size=(3, 3))(x)
    x = Conv2D(128, kernel_size=(3, 3), activation='relu')(x)
    x = BatchNormalization()(x)

    x = Conv2D(128, kernel_size=(3, 3), activation='relu')(x)
    x = BatchNormalization()(x)

    x = MaxPooling2D(pool_size=(3, 3))(x)

    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.25)(x)
    x = Dense(64, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.4)(x)
    x = Dense(num_classes, activation='softmax')(x)
    model = Model(model_input, x, name='model_2')

    return model

In [4]:
model_input_mel = Input(shape=(128,216,1))

In [5]:
model_mel_1 = create_keras_model_mel_1(model_input_mel,50)
model_mel_1.load_weights('../input/mel-spectrogram-training-architectures/mel_model_1.h5')

In [6]:
model_mel_2 = create_keras_model_mel_2(model_input_mel,50)
model_mel_2.load_weights('../input/mel-spectrogram-training-architectures/mel_model_2.h5')

In [7]:
ensemble_models = [model_mel_1,model_mel_2]

In [8]:
def ensemble(models,model_input):
    
    outputs = [model.outputs[0] for model in models]
    y = Average()(outputs)
    
    model = Model(model_input, y, name='ensemble')
    
    return model


In [9]:
ensemble_model = ensemble(ensemble_models,model_input_mel)

In [10]:
ensemble_model.summary()

Model: "ensemble"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 216, 1) 0                                            
__________________________________________________________________________________________________
conv2d_4 (Conv2D)               (None, 126, 214, 64) 640         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 126, 214, 64) 256         conv2d_4[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_4 (MaxPooling2D)  (None, 42, 71, 64)   0           batch_normalization[0][0]        
___________________________________________________________________________________________

**Importing mfcc model**

In [11]:
def create_keras_model_mfcc(model_input, num_classes):

    x = Conv2D(64, kernel_size=(3, 3), activation='relu',padding='same')(model_input)
    x = BatchNormalization()(x)

    x = Conv2D(128, kernel_size=(3, 3), activation='relu',padding='same')(x)
    x = BatchNormalization()(x)
    
    x = MaxPooling2D(pool_size=(4, 4))(x)
    x = Dropout(0.25)(x)
    
    x = Conv2D(128, kernel_size=(3, 3), activation='relu',padding='same')(x)
    x = BatchNormalization()(x)

    x = MaxPooling2D(pool_size=(4, 4))(x)
    x = Dropout(0.25)(x)

    x = Flatten()(x)

    x = Dense(512, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.25)(x)
    
    x = Dense(num_classes, activation='softmax')(x)
    model = Model(model_input,x,name = 'model_2')
    return model

In [12]:
mfcc_model_input = Input(shape=(20,216,1))

In [13]:
mfcc_model = create_keras_model_mfcc(mfcc_model_input,50)
mfcc_model.load_weights('../input/mfcc-spec-training-phase-trial/mfcc_model_2.h5')

In [14]:
mfcc_model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 20, 216, 1)]      0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 20, 216, 64)       640       
_________________________________________________________________
batch_normalization_5 (Batch (None, 20, 216, 64)       256       
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 20, 216, 128)      73856     
_________________________________________________________________
batch_normalization_6 (Batch (None, 20, 216, 128)      512       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 5, 54, 128)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 5, 54, 128)        0   

**Creating a multiple data model**

In [15]:
def multi_model(models,model_input_1,model_input_2):    
    outputs = [model.outputs[0] for model in models]
    y = Average()(outputs)
    model = Model(inputs=[model_input_1,model_input_2],outputs = y, name='multi_model')   
    return model

In [16]:
model = multi_model([ensemble_model,mfcc_model],model_input_mel,mfcc_model_input)

In [17]:
model.summary()

Model: "multi_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 216, 1) 0                                            
__________________________________________________________________________________________________
conv2d_4 (Conv2D)               (None, 126, 214, 64) 640         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 126, 214, 64) 256         conv2d_4[0][0]                   
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 20, 216, 1)] 0                                            
________________________________________________________________________________________

In [18]:
mel_test_data_cat = np.load('../input/test-data-esc-50/mel_test_data_cat.npy',allow_pickle=True)

In [19]:
x_test_mel = np.zeros((1,128,216))
for i in tqdm(range(200)):
    x_test_mel = np.append(x_test_mel,[mel_test_data_cat[i][0][:,:,-1]],axis=0)

100%|██████████| 200/200 [00:01<00:00, 141.80it/s]


In [20]:
x_test_mel = x_test_mel[1:,:,:]
x_test_mel = x_test_mel[:,:,:,np.newaxis]

In [21]:
y_test_mel = []
for i in range(200):
    y_test_mel.append(mel_test_data_cat[i][1])
y_test_mel = np.array(y_test_mel)

In [22]:
mfcc_test_data_cat = np.load('../input/test-data-esc-50/mfcc_test_data_cat.npy',allow_pickle=True)

In [23]:
x_test_mfcc = np.zeros((1,20,216))
for i in tqdm(range(200)):
    x_test_mfcc = np.append(x_test_mfcc,[mfcc_test_data_cat[i][0][:,:,-1]],axis=0)

100%|██████████| 200/200 [00:00<00:00, 1375.03it/s]


In [24]:
x_test_mfcc = x_test_mfcc[1:,:,:]
x_test_mfcc = x_test_mfcc[:,:,:,np.newaxis]

In [25]:
y_test_mfcc = []
for i in range(200):
    y_test_mfcc.append(mfcc_test_data_cat[i][1])
y_test_mfcc = np.array(y_test_mfcc)

In [26]:
arr = model.predict([x_test_mel,x_test_mfcc])

In [27]:
y_pred = []
for i in range(200):
    y_pred.append(np.where(arr[i] == np.amax(arr[i]))[0])

In [28]:
np.unique(y_pred)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49])

In [29]:
y_pred = np.array(y_pred).flatten()

In [30]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_pred,y_test_mel))

0.92


In [31]:
model.save('multi_model.h5')